### Intro

**CheXNet** [[1]](https://arxiv.org/pdf/1711.05225.pdf) is a 121 layer **DenseNet** developed by Stanford researchers that can detect pneumonia from chest X-rays at a level exceeding practicing radiologists. 

The weights of the model are uploaded into this notebook and used to train on our data to classify normal vs opacity (typical, atypical, indeterminate) cases. 

Contrast Limited Adaptive Histogram Equalization (**CLAHE**) is used for preprocessing and some augmentation techniques are applied. 
For interpretability, **GRAD-CAM** is used to see if the model is paying attention to the opacities (comparing to the groundtruth bounding boxes).

### nb work done on top of:

https://www.kaggle.com/sinamhd9/chexnet-fine-tuned-model-interpretation

### get data - 

start from the eda notebook

In [ ]:
import pandas as pd


In [ ]:
study_df = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")
study_df["id"] = study_df["id"].str.replace("_study", "")


In [ ]:
study_df.head()

In [ ]:
import os


In [ ]:
def get_absolute_file_paths(directory):
    all_abs_file_paths = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            all_abs_file_paths.append(os.path.abspath(os.path.join(dirpath, f)))
    return all_abs_file_paths

In [ ]:
from tqdm.notebook import tqdm; tqdm.pandas();


In [ ]:
    study_df["study_dir"] = "/kaggle/input/siim-covid19-detection/train/"+study_df["id"]
    study_df["images_per_study"] = study_df.study_dir.progress_apply(lambda x: len(get_absolute_file_paths(x)))
#     study_df["images_per_study"] = study_df.study_dir.apply(lambda x: len(get_absolute_file_paths(x)))


In [ ]:
study_df.head()

In [ ]:
image_df = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")


In [ ]:
image_df.head()

In [ ]:
image_study_df = pd.merge(image_df, 
                          study_df, 
                          left_on='StudyInstanceUID',
                          right_on='id')

In [ ]:
image_study_df.head()

In [ ]:
image_study_df.shape

In [ ]:
# df_image = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
# df_study['id'] = df_study['id'].str.replace('_study',"")
# df_study.rename({'id': 'StudyInstanceUID'},axis=1, inplace=True)
# df_train = df_image.merge(df_study, on='StudyInstanceUID')


### in this nb, we use the jpg variant of data

### attempt1: image binary classifier - opacity vs. none

In [ ]:
image_study_df.head()

In [ ]:
image_study_df.loc[image_study_df['Negative for Pneumonia']==1, 'study_label'] = 'negative'
image_study_df.loc[image_study_df['Typical Appearance']==1, 'study_label'] = 'typical'
image_study_df.loc[image_study_df['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
image_study_df.loc[image_study_df['Atypical Appearance']==1, 'study_label'] = 'atypical'


In [ ]:
image_study_df.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)


In [ ]:
image_study_df.head(20)

In [ ]:
image_study_df['id_jpg'] = image_study_df['id_x'].str.replace('_image', '.jpg')
image_study_df['image_label'] = image_study_df['label'].str.split().apply(lambda x : x[0])


In [ ]:
image_study_df.head(20)

In [ ]:
image_study_df.image_label.value_counts()

In [ ]:
# df_size = pd.read_csv('../input/covid-jpg-512/size.csv')
# df_train = df_train.merge(df_size, on='id')


In [ ]:
# df_size.head()

In [ ]:
# df_size.split.value_counts()

In [ ]:
# import numpy as np
# import os
# from skimage import exposure
# import matplotlib
# matplotlib.rcParams.update({'font.size': 16})
# import warnings
# warnings.filterwarnings('ignore')
# import tensorflow.keras.backend as K


In [ ]:

# def preprocess_image(img):
#     equ_img = exposure.equalize_adapthist(img/255, clip_limit=0.05, kernel_size=24)
#     return equ_img

# df_opa = df_train[df_train['image_label']=='opacity'].reset_index()
# fig, axs = plt.subplots(5, 2, figsize=(10,20))
# fig.subplots_adjust(hspace=.2, wspace=.2)
# n=5
# for i in range(n):
#     img = cv2.imread(os.path.join(train_dir, df_opa['id'][i]))
#     img_proc = preprocess_image(img)
#     axs[i, 0].imshow(img)
#     axs[i, 1].imshow(img_proc)
#     axs[i, 0].axis('off')
#     axs[i, 1].axis('off')
#     boxes = literal_eval(df_opa['boxes'][i])
#     for box in boxes:
#         axs[i, 0].add_patch(Rectangle((box['x']*(512/df_opa['dim1'][i]), box['y']*(512/df_opa['dim0'][i])), box['width']*(512/df_opa['dim1'][i]), box['height']*(512/df_opa['dim0'][i]), fill=0, color='y', linewidth=3))
#         axs[i, 0].set_title(df_opa['study_label'][i])
#         axs[i, 1].add_patch(Rectangle((box['x']*(512/df_opa['dim1'][i]), box['y']*(512/df_opa['dim0'][i])), box['width']*(512/df_opa['dim1'][i]), box['height']*(512/df_opa['dim0'][i]), fill=0, color='r', linewidth=3))
#         axs[i, 1].set_title('After CLAHE')
    
# plt.show()

## ImageGenerators and Augmentations

In [ ]:
# batch_size = 32
batch_size = 16


In [ ]:
train_dir = '../input/covid-jpg-512/train'


In [ ]:
! ls '../input/covid-jpg-512/train' | wc -l

In [ ]:
! ls -ahl '../input/covid-jpg-512/train' | grep 000c3a3f293f

In [ ]:
import cv2
# img = cv2.imread('../input/covid-jpg-512/train/000a312787f2.jpg')
img = cv2.imread('../input/covid-jpg-512/train/000c3a3f293f.jpg')


In [ ]:
img.shape

In [ ]:
# img_size = 150
img_size = 512


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_generator = ImageDataGenerator(
        rescale = 1./255,
        validation_split=0.25,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=False,
        fill_mode='nearest',
)

image_generator_valid = ImageDataGenerator(validation_split=0.25,
                                           rescale = 1./255,  )


In [ ]:
train_generator = image_generator.flow_from_dataframe(
        dataframe = image_study_df,
        directory='../input/covid-jpg-512/train',
        x_col = 'id_jpg',
        y_col =  'image_label',  
        target_size=(img_size, img_size),
        batch_size=batch_size,
#         class_mode='binary',
        subset='training', 
        seed = 23) 

valid_generator=image_generator_valid.flow_from_dataframe(
    dataframe = image_study_df,
    directory='../input/covid-jpg-512/train',
    x_col = 'id_jpg',
    y_col = 'image_label',
    target_size=(img_size, img_size),
    batch_size=batch_size,
#     class_mode='binary',
    subset='validation', 
    shuffle=False, 
    seed=23) 


In [ ]:
len(valid_generator.filenames)/valid_generator.batch_size

In [ ]:
len(train_generator.filenames)/train_generator.batch_size

In [ ]:
valid_generator.batch_size, valid_generator.class_indices

In [ ]:
# valid_generator.allowed_class_modes
# valid_generator.class_mode
# valid_generator.classes
# valid_generator.color_mode
# valid_generator.directory
# valid_generator.data_format
# valid_generator.dtype
valid_generator.filenames[:5]

In [ ]:
# valid_generator.filepaths[:5]
# valid_generator.image_shape
# valid_generator.labels[:5]
valid_generator.target_size


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# for j in range(4):
#     aug_images = [train_generator[0][0][j] for i in range(5)]
#     fig, axes = plt.subplots(1, 5, figsize=(24,24))
#     axes = axes.flatten()
#     for img, ax in zip(aug_images, axes):
#         ax.imshow(img)
#         ax.axis('off')
# plt.tight_layout()
# plt.show()

## Architecture

In [ ]:
chex_weights_path = '../input/chexnet-weights/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'


In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
# from tensorflow.keras import models


In [ ]:
img_size

In [ ]:
pre_model = DenseNet121(weights=None,
                        include_top=False,
                        input_shape=(img_size,img_size,3)
                        )
out = Dense(14, activation='sigmoid')(pre_model.output)
pre_model = Model(inputs=pre_model.input, outputs=out) 


In [ ]:
pre_model.load_weights(chex_weights_path)


In [ ]:
# pre_model.trainable = False
pre_model.trainable = True



In [ ]:
pre_model.summary()

In [ ]:
len(pre_model.layers)

In [ ]:
pre_model.layers[420:]

In [ ]:
pre_model.layers[-2]

In [ ]:
# last_layer = pre_model.get_layer('conv5_block16_concat')
last_layer = pre_model.layers[-2]

print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
last_layer

In [ ]:
# Flatten the output layer to 1 dimension
x = Flatten()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.2
x = Dropout(0.2)(x)                  
# # Add a fully connected layer with 128 hidden units and ReLU activation
# x = Dense(128, activation='relu')(x)


# Add final classification layer
# x = Dense(1, activation='sigmoid')(x)  
x = Dense(2, activation='softmax')(x)


In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop


In [ ]:
model = Model( pre_model.input, x) 

# model.compile(optimizer = RMSprop(lr=0.01), 
#               loss='binary_crossentropy', 
#               metrics=['accuracy'])

model.compile(Adam(lr=1e-3),
              loss='binary_crossentropy',
              metrics='accuracy')
model.summary()

In [ ]:
train_generator

In [ ]:
valid_generator

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


In [ ]:
rlr = ReduceLROnPlateau(monitor = 'val_accuracy', 
                        factor = 0.2, 
                        patience = 2, 
                        verbose = 1, 
                        min_delta = 1e-4, 
                        min_lr = 1e-4, 
                        mode = 'max')

In [ ]:
es = EarlyStopping(monitor = 'val_accuracy', 
                   min_delta = 1e-4, 
                   patience = 3, 
                   mode = 'max', 
                   restore_best_weights = True, 
                   verbose = 1)


In [ ]:
ckp = ModelCheckpoint('model.h5',
                      monitor = 'val_accuracy',
                      verbose = 0, 
                      save_best_only = True, 
                      mode = 'max')

In [ ]:
history = model.fit(
      train_generator,
      validation_data=valid_generator,
#       steps_per_epoch = 296,
      epochs=10,
      callbacks=[es, ckp, rlr],
#       validation_steps=98,
#       verbose=1
)

### if acc does not change, try these ???:

https://stackoverflow.com/questions/37213388/keras-accuracy-does-not-change

https://datascience.stackexchange.com/questions/30930/accuracy-and-loss-dont-change-in-cnn-is-it-over-fitting

https://www.kaggle.com/questions-and-answers/56171

https://github.com/keras-team/keras/issues/2711


### Things to consider from https://arxiv.org/pdf/1711.05225.pdf:

optimize the weighted binary cross entropy loss

minibatches of size 16. 

pick the model with the lowest validation loss - done.

Learning rate changes - done

better test-train split based on Y balance

normalize based on the mean and standard deviation of images

augment the training data with random horizontal flipping.




## Model performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
import numpy as np

In [ ]:
# from tensorflow.math import confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix


In [ ]:
actual =  valid_generator.labels
preds = np.argmax(model.predict(valid_generator), axis=1)
cfmx = confusion_matrix(actual, preds)
acc = accuracy_score(actual, preds)

In [ ]:
model.evaluate(valid_generator)

In [ ]:
actual[:10]

In [ ]:
preds[:10]

In [ ]:
valid_generator.class_indices

In [ ]:
cfmx

In [ ]:
acc

In [ ]:
# from seaborn import heatmap


In [ ]:
# print ('Test Accuracy:', acc )
# heatmap(cfmx, annot=True, cmap='plasma',
#         xticklabels=['Normal','Opacity'],fmt='.0f', yticklabels=['Normal', 'Opacity'])
# plt.show()

In [ ]:
# hist = pd.DataFrame(history.history)
# fig, (ax1, ax2) = plt.subplots(figsize=(12,12),nrows=2, ncols=1)
# hist['loss'].plot(ax=ax1,c='k',label='training loss')
# hist['val_loss'].plot(ax=ax1,c='r',linestyle='--', label='validation loss')
# ax1.legend()
# hist['accuracy'].plot(ax=ax2,c='k',label='training accuracy')
# hist['val_accuracy'].plot(ax=ax2,c='r',linestyle='--',label='validation accuracy')
# ax2.legend()
# plt.show()

## Model Interpretation 

In [ ]:
def grad_cam(input_image, model, layer_name):

    desired_layer = model.get_layer(layer_name)
    grad_model = Model(model.inputs, [desired_layer.output, model.output])

    with tf.GradientTape() as tape:
        layer_output, preds = grad_model(input_image)
        ix = (np.argsort(preds, axis=1)[:, -1]).item()
        output_idx = preds[:, ix]

    gradient = tape.gradient(output_idx, layer_output)
    alpha_kc = np.mean(gradient, axis=(0,1,2))
    L_gradCam = tf.nn.relu(np.dot(layer_output, alpha_kc)[0])
    L_gradCam = (L_gradCam - np.min(L_gradCam)) / (np.max(L_gradCam) - np.min(L_gradCam)) 
    return L_gradCam.numpy()

In [ ]:
import cv2


In [ ]:
def blend(img_path, gradCam_img, alpha, colormap = cv2.COLORMAP_JET):
    origin_img = img_to_array(load_img(img_path))
    gradCam_resized = cv2.resize(gradCam_img, (origin_img.shape[1], origin_img.shape[0]), interpolation = cv2.INTER_LINEAR)
    heatmap  = cv2.applyColorMap(np.uint8(gradCam_resized*255), colormap)
    superimposed_image = cv2.cvtColor(origin_img.astype('uint8'), cv2.COLOR_RGB2BGR) + heatmap * alpha
    return heatmap, superimposed_image

In [ ]:
def plot_results(model, gen, label=0):
    n = 50
    fig, axs = plt.subplots(10, 5, figsize=(20,60))
    fig.subplots_adjust(hspace=.5, wspace=.1)
    axs = axs.ravel()
    gen.next()
    classes = list(gen.class_indices.keys()) 
    if label==0:
        idx = np.array(np.where(np.array(gen.labels) ==0)).ravel()
    else:
        idx = np.array(np.where(np.array(gen.labels) ==1)).ravel()
   
    layer_name = 'bn'
    for i in range(n):
        sample_img_path = os.path.join(train_dir, df_train['id_jpg'][idx[i]])
        img = load_process(sample_img_path, img_size)
        pred = model.predict(img)
        grad_cam_img = grad_cam(img, model, layer_name)
        heatmap_img, result_img = blend(sample_img_path, grad_cam_img, 0.5)
        axs[i].imshow(result_img[:,:,::-1]/255)
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        if type(df_train['boxes'][idx[i]])==str:
            boxes = literal_eval(df_train['boxes'][idx[i]])
            for box in boxes:
#                 axs[i].add_patch(Rectangle((box['x']*(512/df_train['dim1'][idx[i]]), box['y']*(512/df_train['dim0'][idx[i]])), box['width']*(512/df_train['dim1'][idx[i]]), box['height']*(512/df_train['dim0'][idx[i]]), fill=0, color='y', linewidth=2))
                axs[i].set_title(f"{df_train['study_label'][idx[i]]}, {df_train['image_label'][idx[i]]}")
        else:
            axs[i].set_title(df_train['study_label'][idx[i]])
        
        axs[i].set_xlabel(f"{classes[np.argmax(pred)]}, {round(pred[0][np.argmax(pred)]*100, 2)}%")

In [ ]:
df_train = image_study_df

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:
def load_process(img, img_size):
    img = load_img(img, target_size = (img_size, img_size))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
#     img = preprocess_image(img)
    return img

In [ ]:
import tensorflow as tf


In [ ]:
from ast import literal_eval


In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
# plot_results(model, valid_generator,label=0)

In [ ]:
# plot_results(model, valid_generator,label=1)